In [2]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import filters
import scipy.ndimage as sci_image
from skimage import measure as meas
import matplotlib.pyplot as plt
from scipy import signal as sci_signal
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import os
import warnings
warnings.filterwarnings("ignore")
import supported_functions as sf

In [ ]:
image = cv2.imread('C:\Users\amith\')